# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
import sys
import os

# Make sure we can import the aimakerspace package
if 'aimakerspace' not in sys.modules:
	# Get the absolute path to the parent directory containing the aimakerspace package
	project_root = os.path.abspath(os.path.join(os.path.dirname('__file__')))
	if project_root not in sys.path:
		sys.path.append(project_root)
	print(f"Added {project_root} to sys.path")

#from dotenv import load_dotenv
#load_dotenv(override=True)	

Added /home/mafzaal/source/AIE6/02_Embeddings_and_RAG to sys.path


In [2]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [4]:
text_loader = TextFileLoader("data/llms-full.txt")
documents = text_loader.load_documents()
len(documents)

1

In [5]:
print(documents[0][:100])

<SYSTEM>This is the full developer documentation for Svelte and SvelteKit.</SYSTEM>

# Start of Svel


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

1108

Let's take a look at some of the documents we've managed to split.

In [7]:
split_documents[0:1]

["<SYSTEM>This is the full developer documentation for Svelte and SvelteKit.</SYSTEM>\n\n# Start of Svelte documentation\n\n\n# Overview\n\nSvelte is a framework for building user interfaces on the web. It uses a compiler to turn declarative components written in HTML, CSS and JavaScript...\n\n```svelte\n<!--- file: App.svelte --->\n<script>\n\tfunction greet() {\n\t\talert('Welcome to Svelte!');\n\t}\n</script>\n\n<button onclick={greet}>click me</button>\n\n<style>\n\tbutton {\n\t\tfont-size: 2em;\n\t}\n</style>\n```\n\n...into lean, tightly optimized JavaScript.\n\nYou can use it to build anything on the web, from standalone components to ambitious full stack apps (using Svelte's companion application framework, [SvelteKit](../kit)) and everything in between.\n\nThese pages serve as reference documentation. If you're new to Svelte, we recommend starting with the [interactive tutorial](/tutorial) and coming back here when you have questions.\n\nYou can also try Svelte online in the [

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [8]:
import os
import openai
from getpass import getpass

openai.api_key = os.getenv("OPENAI_API_KEY",getpass("OpenAI API Key: "))
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension? Yes, we can use dimensions parameter, I have modified `async_get_embeddings` method of `EmbeddingModel()` to accept dimensions parameter
2. What technique does OpenAI use to achieve this? OpenAI has trained there embedding models  with dimensionality reduction technique called [Matryoshka Representation Learning](https://arxiv.org/abs/2205.13147).

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [9]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings? Using async will run tasks concurrently in event loop, when one task awaitng on I/O or network then next task will run. As communication with OpenAI is network bound that's why running async will result in better throughput.


> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [11]:
#vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)
vector_db.search_by_text("What is rune and what are different types of rune?", k=3)

[(' = \'#ff3e00\';\n>\n> \tlet canvas;\n>\n> \t$: {\n> \t\tconst context = canvas.getContext(\'2d\');\n> \t\tcontext.clearRect(0, 0, canvas.width, canvas.height);\n>\n> \t\t// this will re-run whenever `color` or `size` change\n> \t\tcontext.fillStyle = color;\n> \t\tcontext.fillRect(0, 0, size, size);\n> \t}\n> </script>\n>\n> <canvas bind:this={canvas} width="100" height="100" />\n> ```\n>\n> This only worked at the top level of a component.\n\n# What are runes?\n\n> [!NOTE] **rune** /ruːn/ _noun_\n>\n> A letter or mark used as a mystical or magic symbol.\n\nRunes are symbols that you use in `.svelte` and `.svelte.js`/`.svelte.ts` files to control the Svelte compiler. If you think of Svelte as a language, runes are part of the syntax — they are _keywords_.\n\nRunes have a `$` prefix and look like functions:\n\n```js\nlet message = $state(\'hello\');\n```\n\nThey differ from normal JavaScript functions in important ways, however:\n\n- You don\'t need to import them — they are part of 

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs? We can use temprature parameter and lower value will help to achieve more reproducible outputs. We can pass `temperature` parameter to `run` method of `ChatOpenAI()`.

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [12]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages, temperature=0.5)

In [13]:
print(response)

The best way to write a loop in Python often depends on the specific task you're trying to accomplish. However, here are some general guidelines and examples for writing efficient and readable loops:

### 1. Using `for` Loops

`for` loops are great for iterating over sequences (like lists, tuples, or strings).

```python
# Example: Iterating over a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

### 2. Using `while` Loops

`while` loops are useful when you want to repeat an action until a certain condition is met.

```python
# Example: Using a while loop
count = 0
while count < 5:
    print(count)
    count += 1
```

### 3. Using List Comprehensions

For simple loops that generate lists, list comprehensions can be a more concise and readable option.

```python
# Example: Creating a list of squares
squares = [x**2 for x in range(10)]
print(squares)
```

### 4. Using `enumerate()` for Index and Value

If you need both the index and the value from a 

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [14]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response? We can instruct LLM to use **Think though your response step by step.** to have more thoughtful, detailed response.

What is that strategy called? The strategy called **Chain of Thought Prompting**

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

In [16]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [18]:
#retrieval_augmented_qa_pipeline.run_pipeline("What is the Michael Eisner Memorial Weak Executive Problem?")
content = retrieval_augmented_qa_pipeline.run_pipeline("What is rune and what are different types of rune?")


In [ ]:
from IPython.display import Markdown
 
# Display the response from the RAG pipeline

Markdown(content['response'])

A **rune** is a symbol used in Svelte, which serves as a keyword for controlling reactivity and providing instructions to the Svelte compiler. They are part of Svelte's syntax and have a `$` prefix, resembling function calls. You don't need to import runes; they are inherent to the Svelte language.

Different types of runes include:

1. **$state**: Used for declaring reactive state.
2. **Derived state** and **side effects**: Managed through various other runes, though specific examples are not provided in the context.

The text does not specify additional types of runes, but it mentions that runes are part of a powerful set of primitives for controlling reactivity in Svelte components.

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [41]:
from collections import defaultdict
import numpy as np
from typing import List, Tuple, Dict, Any, Optional
import numpy as np

# implemented euclidean distance function
def euclidean_distance(vector_a: np.array, vector_b: np.array) -> float:
    return  -1 * (np.linalg.norm(vector_a - vector_b))
        
        

# 1. Create enhanced RAG pipeline with improved prompt and new distance metric support
class EnhancedRAGPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        
        # Enhanced system prompt with Chain of Thought reasoning
        self.system_prompt = SystemRolePrompt("""
        Use the provided context to answer the user's query.
        
        Think through your answer step by step:
        1. First, analyze the relevant information from the provided context
        2. Identify the key points that address the user's query
        3. Organize this information into a clear, structured response
        4. If multiple pieces of context provide different perspectives, synthesize them
        
        You may not answer the user's query unless there is specific context in the following text.
        If you do not know the answer, or cannot answer, please respond with "I don't know".
        
        Include citations from the context when appropriate by referring to the specific text.
        """)
        
        self.user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)
    
    def run_pipeline(self, user_query: str, k: int = 4, metric: str='cosine') -> dict:
        """
        Run the enhanced RAG pipeline with choice of similarity metric
        
        Args:
            user_query: The user's question
            k: Number of context items to retrieve
            metric: "cosine" or "euclidean" for distance calculation
        """
        context_list = []

        if metric == 'euclidean':
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k, distance_measure=euclidean_distance)
        else:
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        # Format context, including any metadata if available
        context_prompt = ""
        for text,_ in context_list:
            
            context_prompt += f"{text}\n\n"
                
        formatted_system_prompt = self.system_prompt.create_message()
        formatted_user_prompt = self.user_prompt.create_message(
            user_query=user_query, 
            context=context_prompt
        )
        
        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt], temperature=0.2),
            "context": context_list
        }



# Create enhanced RAG pipeline
enhanced_rag_pipeline = EnhancedRAGPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)



In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Create a slider for selecting 'k' (number of results to retrieve)
k_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=10,
    step=1,
    description='Top K:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    tooltip='Select number of context chunks to retrieve',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px')
)

# Create input widgets
question_input = widgets.Text(
    value='',
    placeholder='Enter your question here...',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

search_button = widgets.Button(
    description='Get Answer',
    button_style='primary',
    tooltip='Click to search for an answer',
    icon='search'
)

output_area = widgets.Output()

# Create a dropdown for distance metric selection
metric_dropdown = widgets.Dropdown(
    options=['cosine', 'euclidean'],
    value='cosine',
    description='Metric:',
    disabled=False,
)

# Handle button click
def on_button_click(b):
    with output_area:
        clear_output()
        print("Searching for answer... Please wait.")
        
        # Get the question from the input
        question = question_input.value
        
        if not question.strip():
            print("Please enter a question!")
            return
            
        # Get selected metric
        metric = metric_dropdown.value
            
        # Get response using the RAG pipeline
        
        result = enhanced_rag_pipeline.run_pipeline(question, k=k_slider.value, metric=metric)
        
        # Display the response nicely
        clear_output()
        display(Markdown("### Answer:"))
        display(Markdown(result['response']))
        
        # Display context sources
        display(Markdown("### Sources:"))
        for i, (text, score) in enumerate(result['context']):
            preview = text[:150].replace('\n', ' ') + "..." if len(text) > 150 else text
            display(Markdown(f"**Source {i+1}** (similarity score: {score:.4f}):\n```\n{preview}\n```"))

# Connect the button click event to the handler
search_button.on_click(on_button_click)

# Layout the widgets
header = widgets.HTML("<h2>RAG Question Answering System</h2>")
widgets_container = widgets.VBox([
    header,
    widgets.HBox([question_input, k_slider,metric_dropdown]),
    search_button,
    output_area
])

# Display the widget
display(widgets_container)